In [1]:
%%capture --no-stderr
!pip install langchain langchain-openai langchain-core langchain-community
!pip install langsmith

In [123]:
import os
import json
from enum import Enum
from google.colab import userdata
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field, field_validator

In [3]:
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_KEY')
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGSMITH_KEY')
os.environ["LANGCHAIN_PROJECT"] = "notebook-creation-langchain"

In [4]:
llm = ChatOpenAI(model="gpt-4.1-nano")

In [10]:
section_topics = """
**Introdução ao NumPy e Pandas**
   - Apresentação das bibliotecas NumPy e Pandas
   - Arrays NumPy e operações vetorizadas
   - Estruturas de dados: Series e DataFrame
   - Leitura de arquivos CSV
   - Indexação, filtros e operações com colunas"""

In [11]:
section_topics_prompt = """
#################### Contexto ####################

Você é um assistente para a geração de conteúdo, seu papel é separar todos
os tópicos do conteúdo de uma seção apresentada no texto a seguir em uma lista de tópicos.

{section_topics}
"""

In [99]:
class Topic(BaseModel):
    text: str = Field(description='content of topic text string')

class SubTopic(BaseModel):
    text: str = Field(description='content of subtopic text string')

class SplittedSubTopic(BaseModel):
    new_subtopics: list[SubTopic] = Field(description="organized subtopic content")

In [67]:
class SectionContent(BaseModel):
    topics: list[Topic] = Field(description="list of topics in the section")

In [68]:
section_topics_chain = ChatPromptTemplate.from_template(section_topics_prompt) | llm.with_structured_output(SectionContent)

In [69]:
section_topics_response = section_topics_chain.invoke(
    {
        "section_topics": section_topics
    }
)

In [71]:
section_topics_response.topics[0].text

'Apresentação das bibliotecas NumPy e Pandas'

In [88]:
subtopics_prompt = """
#################### Contexto ####################

Você é um assistente para a geração de conteúdo, seu papel é explicar em detalhes
o tópico apresentado a seguir para que um notebook seja criado explicando
posteriormente o conteúdo.

Explique detalhadamente o seguinte tópico:
{topic}

Siga as seguintes instruções:
- Apresente apenas o conteúdo de forma clara
- Não faça conclusões sobre o conteúdo criado
- Não faça interação com o leitor do conteúdo
- Se o conteúdo for para introdução de algum tema, não será necessário gerar
 código em Python para exemplificar
"""

In [89]:
class TopicTextContent(BaseModel):
    text: str = Field(description="full content explaining the topic")

In [90]:
subtopics_chain = ChatPromptTemplate.from_template(subtopics_prompt) | llm.with_structured_output(TopicTextContent)

In [94]:
subtopics_response = subtopics_chain.invoke(
    {
        "topic": section_topics_response.topics[1].text
    }
)

In [93]:
section_topics_response.topics[1].text

'Arrays NumPy e operações vetorizadas'

In [95]:
print(subtopics_response.text)

Arrays NumPy são estruturas de dados multidimensionais altamente eficientes para armazenamento e manipulação de grandes conjuntos de dados numéricos em Python. São semelhantes às listas, mas oferecem vantagens significativas em termos de desempenho e funcionalidade, principalmente no contexto de operações matemáticas e científicas.

Operações vetorizadas referem-se ao processamento de arrays NumPy usando operações elementares que atuam simultaneamente sobre todos os elementos do array, ao contrário de loops tradicionais. Essa abordagem aproveita otimizações de baixo nível e SIMD (Single Instruction, Multiple Data), o que resulta em execução mais rápida e código mais conciso.

Por exemplo, em vez de iterar manualmente por elementos para realizar somas ou multiplicações entre arrays, operações vetorizadas permitem executar tais operações de forma direta e eficiente. Isso é possível porque o NumPy sobrecarrega operadores matemáticos comuns, aplicando-os elemento por elemento de maneira ot

In [96]:
splitted_content_prompt = """
#################### Contexto ####################

Você é um assistente para a geração de conteúdo, seu papel é quebrar o conteúdo
textual de um tópico em uma lista de subtópicos para melhor compreensão do conteúdo.

Organize o conteúdo da melhor maneira possível.

O conteúdo textual a ser organizado é o seguinte:
{textual_content}
"""

In [100]:
splitted_content_chain = ChatPromptTemplate.from_template(splitted_content_prompt) | llm.with_structured_output(SplittedSubTopic)

In [101]:
splitted_content_response = splitted_content_chain.invoke(
    {
        "textual_content": subtopics_response.text
    }
)

In [103]:
splitted_content_response.new_subtopics

[SubTopic(text='Introdução aos Arrays NumPy'),
 SubTopic(text='Vantagens de Arrays NumPy em relação às listas'),
 SubTopic(text='Operações vetorizadas em arrays NumPy'),
 SubTopic(text='Exemplos de operações vetorizadas (soma, subtração, multiplicação, divisão, exponenciação)'),
 SubTopic(text='Funções matemáticas em operações vetorizadas (logaritmos, funções trigonométricas, estatísticas)'),
 SubTopic(text='Uso de broadcasting para operações entre arrays de diferentes formas e tamanhos'),
 SubTopic(text='Aplicações práticas de arrays NumPy e operações vetorizadas')]

In [ ]:
prompt = """
#################### Contexto ####################

Você é um assistente para a geração de conteúdo, seu papel é descrever
no formato de subtópicos os tópicos apresentados a seguir para que uma lista
de exercícios prática seja criada para a fixação do conteúdo da disciplina.

Enumere os principais e mais importantes subtópicos para o conteúdo a seguir:
{topics}
"""

In [ ]:
template = ChatPromptTemplate.from_template(prompt)

In [ ]:
chain = template | llm

In [ ]:
response = chain.invoke(
    {
        "topics": section_topics
    }
)

In [25]:
# print(response.content)

#**Criação do notebook .pynb**

In [104]:
class CellTypes(Enum):
    markdown = 'markdown'
    code = 'code'

In [105]:
class NotebookCell(BaseModel):
    #metadata: dict = Field(description='notebook cell metadata')
    cell_type: CellTypes = Field(description='notebook cell type')

In [106]:
class CodeCell(NotebookCell):
    source: list[str] = Field(
        description='List of Python code lines to exemply the theorical content'
        )

    @field_validator('source')
    @classmethod
    def validate_and_split_strings(cls, value: list[str]) -> list[str]:
        """
        Validates if the input is a list with a single string containing newlines.
        If so, splits the string by newlines and returns the resulting list.
        """
        if len(value) == 1 and '\n' in value[0]:
            return [line + '\n' for line in value[0].split('\n') if len(line) > 0]

        return value


class MarkdownCell(NotebookCell):
    source: list[str] = Field(
        description='List containing topic name and explanation about what it is.'
        )

    @field_validator('source')
    @classmethod
    def validate_and_split_strings(cls, value: list[str]) -> list[str]:
        """
        Validates if the input is a list with a single string containing newlines.
        If so, splits the string by newlines and returns the resulting list.
        """
        if len(value) == 1 and '\n' in value[0]:
            return [line + '\n' for line in value[0].split('\n') if len(line) > 0]

        return value

class Content(BaseModel):
    markdown: MarkdownCell = Field(description='markdown text explaining the content')
    code: CodeCell = Field(description='python code to exemplify the content')

In [125]:
prompt_notebook = """
#################### Contexto ####################

Você é um assistente para a criação de conteúdos didáticos para uma disciplina.
Seu papel é gerar o conteúdo teórico e o código em python para serem adicionados
em um notebook como forma de exemplificar o que o aluno precisa aprender.

Gere sempre um par de células, sendo uma célula em markdown com o conteúdo explicativo
e outra célula com o código python, exemplificando claramente o conteúdo.

Orientações:
- Para as células em markdown adicione sempre um título resumindo o tema abordado em poucas palavras
- Use # para definir o título e coloque em negrito
- A explicação do conteudo coloque como um texto normal, podendo usar listas e/ou bullet point
- Gere o texto de explicação sempre em {language}
- Para as células de código python, gere um código que exemplifique ou explique o tópico abordado
- Todos os códigos deverão ser gerados em inglês e com nomes significativos para as variáveis
- Se algum comentário for feito para explicar o código, adicione um caracter de quebra linha extra para separar
o comentário do código gerado

Gere as células em markdown e código python para o seguinte tópico {topic} que pertence ao tema {content}
"""

In [126]:
prompt_notebook_template = ChatPromptTemplate.from_template(prompt_notebook)

In [127]:
structured_llm = llm.with_structured_output(Content)

In [128]:
notebook_chain = prompt_notebook_template | structured_llm

In [129]:
print(section_topics_response.topics[1].text)
print(splitted_content_response.new_subtopics[1].text)

Arrays NumPy e operações vetorizadas
Vantagens de Arrays NumPy em relação às listas


In [130]:
response = notebook_chain.invoke(
    {
        "language": "pt-BR",
        "content": section_topics_response.topics[1].text,
        "topic": splitted_content_response.new_subtopics[1].text
    }
)

In [131]:
response.markdown.source

['# **Vantagens de Arrays NumPy em relação às listas**\n',
 'Arrays NumPy oferecem várias vantagens em comparação com listas tradicionais do Python, especialmente ao trabalhar com operações matemáticas e científicas:\n',
 '- **Operações vetorizadas:** Permitem realizar operações em toda a matriz de uma só vez, sem a necessidade de loops explícitos.\n',
 '- **Desempenho melhorado:** São otimizados em linguagem C, o que resulta em execuções mais rápidas para operações matemáticas e manipulação de grandes conjuntos de dados.\n',
 '- **Menor consumo de memória:** Utilizam menos memória do que listas tradicionais para armazenar os mesmos dados, devido ao seu formato compacto.\n',
 '- **Funcionalidades específicas:** Oferecem diversas funções matemáticas e estatísticas que facilitam análises complexas.\n',
 'A seguir, um exemplo comparando a realização de uma operação de soma em listas e arrays NumPy.\n']

In [132]:
response.code.source

['import numpy as np\n',
 '# Criando uma lista de números\n',
 'list_numbers = [1, 2, 3, 4, 5]\n',
 '# Criando um array NumPy a partir da lista\n',
 'array_numbers = np.array([1, 2, 3, 4, 5])\n',
 '# Soma de 10 a todos os elementos usando listas (com loop explícito)\n',
 'sum_list = [x + 10 for x in list_numbers]\n',
 '# Soma de 10 a todos os elementos usando NumPy (operações vetorizadas)\n',
 'sum_array = array_numbers + 10\n',
 'print("Soma usando listas:", sum_list)\n',
 'print("Soma usando NumPy:", sum_array)\n']

In [149]:
generated_cells = {
    "cells": [
        {
            "cell_type": "markdown",
            "source":
                response.markdown.source
            ,
            "metadata": {
                "id": ""
            }
        },
        {
            "cell_type": "code",
            "execution_count": None,
            "metadata": {
                "id": ""
            },
            "outputs": [],
            "source": response.code.source
        }
        ]
}

In [ ]:
print(json.dumps(generated_cells["cells"], indent=4, ensure_ascii=False))

In [146]:
import uuid
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [147]:
path = '/content/drive/MyDrive/01 - PUC/2025-01/Python para DS/notebooks'

In [164]:
os.listdir(path)

['notebook_b20c6afedd2a4b96a4321cba0d1c3c7f.ipynb']

In [152]:
notebook = {
  "nbformat": 4,
  "nbformat_minor": 0,
  "metadata": {
    "colab": {
      "provenance": []
    },
    "kernelspec": {
      "name": "python3",
      "display_name": "Python 3"
    },
    "language_info": {
      "name": "python"
    }
  },
  "cells": generated_cells["cells"]
}

In [163]:
with open(
    file=os.path.join(path, f"notebook_{uuid.uuid4().hex}.ipynb"),
    mode="w",
    encoding="utf-8"
    ) as f:

    json.dump(notebook, f, indent=4, ensure_ascii=False)